In [44]:
import pandas as pd
import matplotlib as plt
from collections import Counter

In [50]:
train = pd.read_csv('/Users/hanan/Downloads/kaggle/train.csv')

In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [51]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


train['Cabin_letter'] = [str(c)[0].upper() for c in train['Cabin'].tolist()]

In [197]:
train2 = train[['PassengerId','Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
train2[['Age','SibSp','Parch','Fare']] = train2[['Age','SibSp','Parch','Fare']].apply(pd.to_numeric)
train2[['Sex','Embarked']] = train2[['Sex','Embarked']].fillna('N')

train2._get_numeric_data()

In [198]:
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.preprocessing import Imputer,StandardScaler,OneHotEncoder,LabelBinarizer,LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin

In [199]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [231]:
num_attribs = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [232]:
id_pipeline = Pipeline([
        ('selector', DataFrameSelector(['PassengerId'])),
        ])
num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attribs)),
        ('imputer', Imputer(strategy="median")),
        ('std_scaler', StandardScaler()),
        ])
cat_sex_pipeline = Pipeline([
        ('selector', DataFrameSelector(['Sex'])),
        #('labelEncoder', LabelEncoder()),
        #('OneHotEncoder', OneHotEncoder()),
        ('label_binarizer', LabelBinarizer()),
    ])
cat_embarked_pipeline = Pipeline([
        ('selector', DataFrameSelector(['Embarked'])),
        #('labelEncoder', LabelEncoder()),
        #('OneHotEncoder', OneHotEncoder()),
        ('label_binarizer', LabelBinarizer()),
    ])
full_pipeline = FeatureUnion(transformer_list=[
        #("id_pipeline",id_pipeline),
        ("num_pipeline", num_pipeline),
        ("cat_sex_pipeline", cat_sex_pipeline),
        ("cat_embarked_pipeline", cat_embarked_pipeline),
    ])

In [233]:
train3 = full_pipeline.fit_transform(train2)

In [234]:
train3.shape

(891, 10)

In [235]:
from sklearn.linear_model import LogisticRegression

In [236]:
log_reg = LogisticRegression()
log_reg.fit(train3, train2['Survived'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [240]:
log_reg.get_params

<bound method LogisticRegression.get_params of LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)>